In [1]:
import kfp.dsl as dsl
import kfp
import pandas as pd
import kfp.gcp as gcp
from kfp import components
import json
from string import Template
from config import Config
import os
import kubernetes
import yaml
import sklearn

In [17]:
import subprocess
import sys
a = subprocess.run(["echo", "$GOOGLE_APPLICATION_CREDENTIALS"])
print(a.stdout)

None


In [2]:

train_temp = Template("""
   
    {
  "apiVersion": "batch/v1",
  "kind": "Job",
  "metadata": {
    "name": "sklearn-job",
  },
  "spec": {
    "template": {
      "spec": {
        "restartPolicy": "Never",
        "containers": [
          {
            "name": "sklearn-job",
            "image": "$image",
            "imagePullPolicy": "Always",
            "command": [
              "python",
              "/opt/model.py",
              "--export-dir=$export"
            ],
            
            "env": [
            {
                "name": "GOOGLE_APPLICATION_CREDENTIALS",
                "value": "/var/secrets/user-gcp-json"
            }
            ],
            "volumeMounts": [
            {
                "name": "sa"
                "mountPath": "/var/secrets",
                "readOnly": "true"
            }
            ]
          }
        ],
        "volumes": [
        {
            "name": "sa",
            "secret": {
                "secretName": "user-gcp-sa"
            }
        }
        ]
      }
    }
  }
}

""")


In [3]:
def convert_result(result) -> str:
    import json
    hyperparameters = json.loads(result)
    args = []
    print(type(hyperparameters))
    for key, value in hyperparameters.items():
        args.append("{0}={1}".format(key, value))
        
    return " ".join(args)





# x = '{"name":"John", "age":30, "city":"New York", "value": "none yet"}'
# convert_result(x)       

In [4]:
@dsl.pipeline(
    name="Police-Shootings-Justification-Prediction",
    description="A pipeline to train and serve police shootings justification in the United States"
)

def police_shootings_pipeline(
#     katib_namespace="kubeflow-olawunmi1602",
#     goal=0.9,
    training_image="gcr.io/kubeflow-292422/train-sklearn:latest",
    training_namespace="kubeflow",
#     serving_name="police-shootings-server",
#     serving_namespace="kubeflow",
    export_dir="gs://police-shootings/export/train"
#     serving_export_dir="gs://police-shootings/export/serve",
#     transformer_image="gcr.io/kubeflow-292422/police-shootings-processing:latest"
):
    trainingjson = train_temp.substitute({
        'image': str(training_image),
        'export': str(export_dir)
    })
    
    trainingdeployment = json.loads(trainingjson)
    train_op = dsl.ResourceOp(
        name="train",
        k8s_resource=trainingdeployment,
        action="apply"
#         success_condition="status"
    )
    
    
    
     
    

In [5]:
pipeline = kfp.Client().create_run_from_pipeline_func(police_shootings_pipeline, arguments={})

JSONDecodeError: Expecting property name enclosed in double quotes: line 8 column 3 (char 101)